In [41]:
from psycopg2 import pool

connection_pool = pool.SimpleConnectionPool(
    1, 10,  
    host="13.62.85.98",
    database="ree",
    user="postgres",
    password="postgres"
)

conn = connection_pool.getconn()

connection_pool.putconn(conn)


In [40]:
#cerrar las conexiones
connection_pool.closeall()


In [37]:
data_json = [
    {"id": 1, "nombre": "Juan", "edad": 30},
    {"id": 2, "nombre": "Ana", "edad": 25}
]


In [43]:
from psycopg2 import sql

table_name = "mi_tabla_dinamica"

# Extraer columnas del primer registro del JSON
columns = data_json[0].keys()

# Generar sentencia CREATE TABLE
columns_with_types = ", ".join(f"{col} TEXT" for col in columns)  # todo como TEXT para simplificar
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});"

conn = connection_pool.getconn()
cursor = conn.cursor()

cursor.execute(create_table_query)
conn.commit()
cursor.close()
connection_pool.putconn(conn)


['id', 'nombre', 'edad']


In [39]:
conn = connection_pool.getconn()
cursor = conn.cursor()

for row in data_json:
    cols = row.keys()
    values = [str(row[col]) for col in cols] 
    insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
        sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, cols)),
        sql.SQL(', ').join(sql.Placeholder() * len(cols))
    )
    cursor.execute(insert_query, values)

conn.commit()
cursor.close()
connection_pool.putconn(conn)
